# Client

This section describes how the jupyter client normally works and how to work with different implementations of the jupyter client.

---

The following cell runs the simpliest `IPythonKernlel` that is commonly used for experimenting with jupyter clients.

In [59]:
from pathlib import Path
from utils import run_kernel_in_process
from jupyter_client.blocking import BlockingKernelClient
from IPython.display import JSON


connection_file = Path("/tmp")/"connection_file.json"
connection_file.unlink(missing_ok=True)
run_kernel_in_process(connection_file="/tmp/connection_file.json")

NOTE: When using the `ipython kernel` entry point, Ctrl-C will not work.

To exit, you will have to explicitly quit this process, by either sending
"quit" from a client, or using Ctrl-\ in UNIX-like environments.

To read more about this, see https://github.com/ipython/ipython/issues/2049


To connect another client to this kernel, use:
    --existing /tmp/connection_file.json


## Creating client

The following cell defines the client that we'll use as an example. You need to pass the connection file to the kernel and call the `start_channels` method.

In [ ]:
client = BlockingKernelClient()
client.load_connection_file(connection_file)
client.start_channels()

The conenction can be checked by using the `is_alive` method - it must return `True`.

In [13]:
client.is_alive()

True

## Execute

For execution code, use the `execute` method to point to code that needs to be executed. It returns `msg_id` which allows to identify which messages from the kernel correspond to which messages returned by the kernel.

To get messages from the kernel, use the client's `get_iopub_msg` method of the client. It consistently returns different messages that correspond to different stages of execution. 

---

The following cell sends some code to be executed by the kernel.

In [84]:
msg_id = client.execute("x = 42\ny = x * 2\ny")
msg_id

'99566749-dde045f26306ed8070cf45a5_51308_11'

The following cell collects all messages generaged by the kernel.

In [85]:
ans_list = []

while True:
    try:
        ans_list.append(client.get_iopub_msg(timeout=5))
    except:
        break

In this case 4 messages were returned by the kernel. There is a lot of duplicated information: what's importand at this stage is: 

- `msg_type`: this allows to understand what type of message we got, for example `execute_result` means that this message.
- `content`: the content that the kernel sends to the client - depends on `msg_type`.
- `header.msg_id`: This allows to understand to which message from the client this response belongs to.

In [86]:
ans_list[0]

{'header': {'msg_id': '828386eb-bf56f05325d4274482ffa1df_53484_60',
  'msg_type': 'status',
  'username': 'fedor',
  'session': '828386eb-bf56f05325d4274482ffa1df',
  'date': datetime.datetime(2025, 4, 14, 15, 46, 36, 576434, tzinfo=tzutc()),
  'version': '5.3'},
 'msg_id': '828386eb-bf56f05325d4274482ffa1df_53484_60',
 'msg_type': 'status',
 'parent_header': {'msg_id': '99566749-dde045f26306ed8070cf45a5_51308_11',
  'msg_type': 'execute_request',
  'username': 'fedor',
  'session': '99566749-dde045f26306ed8070cf45a5',
  'date': datetime.datetime(2025, 4, 14, 15, 46, 36, 574804, tzinfo=tzutc()),
  'version': '5.3'},
 'metadata': {},
 'content': {'execution_state': 'busy'},
 'buffers': []}

In [87]:
ans_list[1]

{'header': {'msg_id': '828386eb-bf56f05325d4274482ffa1df_53484_61',
  'msg_type': 'execute_input',
  'username': 'fedor',
  'session': '828386eb-bf56f05325d4274482ffa1df',
  'date': datetime.datetime(2025, 4, 14, 15, 46, 36, 577101, tzinfo=tzutc()),
  'version': '5.3'},
 'msg_id': '828386eb-bf56f05325d4274482ffa1df_53484_61',
 'msg_type': 'execute_input',
 'parent_header': {'msg_id': '99566749-dde045f26306ed8070cf45a5_51308_11',
  'msg_type': 'execute_request',
  'username': 'fedor',
  'session': '99566749-dde045f26306ed8070cf45a5',
  'date': datetime.datetime(2025, 4, 14, 15, 46, 36, 574804, tzinfo=tzutc()),
  'version': '5.3'},
 'metadata': {},
 'content': {'code': 'x = 42\ny = x * 2\ny', 'execution_count': 12},
 'buffers': []}

In [88]:
ans_list[2]

{'header': {'msg_id': '828386eb-bf56f05325d4274482ffa1df_53484_62',
  'msg_type': 'execute_result',
  'username': 'fedor',
  'session': '828386eb-bf56f05325d4274482ffa1df',
  'date': datetime.datetime(2025, 4, 14, 15, 46, 36, 579208, tzinfo=tzutc()),
  'version': '5.3'},
 'msg_id': '828386eb-bf56f05325d4274482ffa1df_53484_62',
 'msg_type': 'execute_result',
 'parent_header': {'msg_id': '99566749-dde045f26306ed8070cf45a5_51308_11',
  'msg_type': 'execute_request',
  'username': 'fedor',
  'session': '99566749-dde045f26306ed8070cf45a5',
  'date': datetime.datetime(2025, 4, 14, 15, 46, 36, 574804, tzinfo=tzutc()),
  'version': '5.3'},
 'metadata': {},
 'content': {'data': {'text/plain': '84'},
  'metadata': {},
  'execution_count': 12},
 'buffers': []}

In [89]:
ans_list[3]

{'header': {'msg_id': '828386eb-bf56f05325d4274482ffa1df_53484_64',
  'msg_type': 'status',
  'username': 'fedor',
  'session': '828386eb-bf56f05325d4274482ffa1df',
  'date': datetime.datetime(2025, 4, 14, 15, 46, 36, 584279, tzinfo=tzutc()),
  'version': '5.3'},
 'msg_id': '828386eb-bf56f05325d4274482ffa1df_53484_64',
 'msg_type': 'status',
 'parent_header': {'msg_id': '99566749-dde045f26306ed8070cf45a5_51308_11',
  'msg_type': 'execute_request',
  'username': 'fedor',
  'session': '99566749-dde045f26306ed8070cf45a5',
  'date': datetime.datetime(2025, 4, 14, 15, 46, 36, 574804, tzinfo=tzutc()),
  'version': '5.3'},
 'metadata': {},
 'content': {'execution_state': 'idle'},
 'buffers': []}